# Demo

This notebook is provided to describe the usage of pydpac using a one-dimensional toy model known as Lorenz96 ([Lorenz 1995](https://www.ecmwf.int/node/10829), [Lorenz and Emanuel 1998](https://doi.org/10.1175/1520-0469%281998%29055%3C0399:OSFSWO%3E2.0.CO;2)) described by the following equation.
$$
\frac{\mathrm{d}X_j}{\mathrm{d}t} = (X_{j+1} - X_{j-2})X_{j-1} - X_j + F
$${#eq-l96}
where $j=1,\cdots,40$ is a grid index and $X_0 = X_{40}$. 

## Available DA algorithms

A data assimilation (DA) algorithm can be chosen from the followings.

- Deterministic DA
    * Kalman filter ([Kalman 1960](https://doi.org/10.1115/1.3662552))
    * 3-dimensional, 4-dimensional variational method (3DVar, 4DVar; [Talagrand and Courtier 1987](https://doi.org/10.1002/qj.49711347812))
- Ensemble DA
    * Ensemble Kalman Filter ([Evensen 1994](https://doi.org/10.1029/94JC00572))
        + Ensemble transform Kalman filter (ETKF; [Bishop et al. 2001](https://doi.org/10.1175/1520-0493%282001%29129%3C0420:ASWTET%3E2.0.CO;2))
        + Perturbed observation method (PO; [Burgers et al. 1998](https://doi.org/10.1175/1520-0493%281998%29126%3C1719:ASITEK%3E2.0.CO;2), [Houtekamer et al.2005](https://doi.org/10.1175/MWR-2864.1))
        + Serial ensemble square root filter (EnSRF; [Whitaker and Hamill 2002](https://doi.org/10.1175/1520-0493%282002%29130%3C1913:EDAWPO%3E2.0.CO;2))
        + Ensemble adjustment Kalman filter (EAKF, local least-square formulation; [Anderson 2003](https://doi.org/10.1175/1520-0493%282003%29131<0634:ALLSFF>2.0.CO;2))
        + Local ensemble transform Kalman filter (LETKF; [Hunt et al. 2007](https://doi.org/10.1016/j.physd.2006.11.008))
    * Maximum likelihood ensemble filter (MLEF; [Zupanski 2005](https://doi.org/10.1175/MWR2946.1), [Zupanski et al. 2008](https://doi.org/10.1002/qj.251))
    * Ensemble variational method (EnVar; [Liu et al. 2008](https://doi.org/10.1175/2008MWR2312.1))

## Requirements

- numpy for the model and DA
- pandas for error statistics
- matplotlib for plots

## Usage

Execute the cells below sequentially.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from model.lorenz import L96
from analysis.obs import Obs

In [ ]:
# model parameter
#model = "l96"

global nx, F, dt

nx = 40     # number of points
F  = 8.0    # forcing
dt = 0.05 / 6  # time step (=1 hour)

# forecast model forward operator
step = L96(nx, dt, F)

In [3]:
# experimental parameter
params = dict()
params["t0off"] =   8 # initial offset between adjacent members
params["t0c"] =    500 # initial time (t0) for control
params["nt"] =     6 # number of step per forecast (=6 hours)
params["na"] =   100 # number of analysis
params["namax"] = 1460 # max number of analysis (1 year)

In [13]:
# observation settings
params["nobs"] = 40 # number of observation (less than or equal to nx)
## op = observation type
params["op"] = "linear"
## observation error
sigma = {"linear": 1.0, "quadratic": 8.0e-1, "cubic": 7.0e-2, \
    "quadratic-nodiff": 8.0e-1, "cubic-nodiff": 7.0e-2, "test":1.0}
ftype = {"mlef":"ensemble","etkf":"ensemble",\
    "po":"ensemble","srf":"ensemble","eakf":"ensemble","letkf":"ensemble",\
        "kf":"deterministic","var":"deterministic","var4d":"deterministic"}

In [32]:
# DA settings
## pt = DA type
params["pt"] = "letkf"
params["nmem"] =   20 # ensemble size
params["linf"] = True # inflation switch (True=Apply, False=Not apply)
params["iinf"] = -1 # inflation type (None=no inflation)
params["infl_parm"] = 1.1 # multiplicative inflation parameter
params["lloc"] = True # localization switch (True=Apply, False=Not apply)
params["lsig"] = 8.0 # localization radius
params["iloc"] = None # localization type (see analysis/mlef.py)
params["ltlm"] = False  # tangent linear operator switch (True=Use, False=Not use)
params["a_window"] = 1 # assimilation window length (for 4-dimensional DA)
params["sigb"] = 0.6 # (for var & 4dvar) background error standard deviation
params["lb"]         = -1.0     # (For var & 4dvar) correlation length for background error covariance

In [ ]:
# module setting
global op, pt, ft
op = params["op"]
pt = params["pt"]
ft = ftype[pt]
params["ft"] = ft

# observation operator
obs = Obs(op, sigma[op])

# specify assimilation method
if pt=='mlef':
    from analysis.mlef import Mlef
    analysis = Mlef(nx, params["nmem"], obs, \
        linf=params["linf"], iinf=params["iinf"], infl_parm=params["infl_parm"], \
        lloc=params["lloc"], iloc=params["iloc"], lsig=params["lsig"], \
        calc_dist=step.calc_dist, calc_dist1=step.calc_dist1,\
        ltlm=params["ltlm"])
elif pt == "envar":
    from analysis.envar import EnVAR
    analysis = EnVAR(nx, params["nmem"], obs, \
        linf=params["linf"], iinf=params["iinf"], infl_parm=params["infl_parm"],\
        lloc=params["lloc"], iloc=params["iloc"], lsig=params["lsig"], \
        calc_dist=step.calc_dist, calc_dist1=step.calc_dist1,\
        ltlm=params["ltlm"])
elif pt == "etkf" or pt == "po" or pt == "letkf" or pt == "srf" or pt=="eakf":
    from analysis.enkf import EnKF
    analysis = EnKF(pt, nx, params["nmem"], obs, \
        linf=params["linf"], iinf=params["iinf"], infl_parm=params["infl_parm"], \
        lloc=params["lloc"], iloc=params["iloc"], lsig=params["lsig"], \
        ltlm=params["ltlm"], \
        calc_dist=step.calc_dist, calc_dist1=step.calc_dist1)
elif pt == "kf":
    from analysis.kf import Kf
    analysis = Kf(obs, 
    infl=params["infl_parm"], linf=params["linf"], 
    step=step, nt=params["nt"])
elif pt == "var":
    from analysis.var import Var
    analysis = Var(obs, 
    sigb=params["sigb"], lb=params["lb"])
elif pt == "4dvar":
    from analysis.var4d import Var4d
    #a_window = 5
    sigb = params["sigb"] * np.sqrt(float(a_window))
    analysis = Var4d(obs, step, params["nt"], a_window,
    sigb=sigb, lb=params["lb"])
elif pt == "4detkf" or pt == "4dpo" or pt == "4dletkf" or pt == "4dsrf":
    from analysis.enkf4d import EnKF4d
    #a_window = 5
    analysis = EnKF4d(pt, nx, params["nmem"], obs, step, params["nt"], a_window, \
        linf=params["linf"], infl_parm=params["infl_parm"], 
        iloc=params["iloc"], lsig=params["lsig"], calc_dist=step.calc_dist, calc_dist1=step.calc_dist1, \
        ltlm=params["ltlm"])
elif pt == "4dmlef":
    #a_window = 5
    from analysis.mlef4d import Mlef4d
    analysis = Mlef4d(nx, params["nmem"], obs, step, params["nt"], a_window, \
            linf=params["linf"], infl_parm=params["infl_parm"], \
            iloc=params["iloc"], lsig=params["lsig"], calc_dist=step.calc_dist, calc_dist1=step.calc_dist1, \
            ltlm=params["ltlm"])

In [34]:
# load functions
from l96_func import L96_func
func = L96_func(step,obs,analysis,params)

In [ ]:
# get truth and make observation
xt, yobs = func.get_true_and_obs()
# initialize all variables
u, xa, xf, pa = func.initialize()
pf = analysis.calc_pf(u, cycle=0)

# analysis-forecast cycle timelot 
# (if analysis is 4-dimensional, analysis is taken every a_window steps.)
na = params["na"]
a_window = params["a_window"]
a_time = range(0, na, a_window)
e = np.zeros(na) # RMSE between truth and analysis
stda = np.zeros(na) # analysis error standard deviation
chi = np.zeros(na) # Chi2 test (values are nearly equal to 1 is good)
dfs = np.zeros(na) # degree of freedom for signal
for i in a_time:
    # read observation
    yloc = yobs[i:i+a_window,:,0]
    y = yobs[i:i+a_window,:,1]
    # analysis
    if pt[:2] == "4d": # assimilate observations at different time (4-dimensional)
        u, pa, spa, innv, chi2, ds = analysis(u, pf, y, yloc, cycle=i)
        for j in range(y.shape[0]):
            chi[i+j] = chi2
            dfs[i+j] = ds
    else: # assimilate observations at a time
        u, pa, spa, innv, chi2, ds = analysis(u, pf, y[0], yloc[0], icycle=i)
        chi[i] = chi2
        dfs[i] = ds
    # restore results
    # analysis
    if ft=="ensemble":
        if pt == "mlef" or pt == "4dmlef":
            xa[i] = u[:, 0]
        else:
            xa[i] = np.mean(u, axis=1)
    else:
        xa[i] = u 
    chi[i] = chi2
    dfs[i] = ds
    if i < na-1:
        # forecast
        if a_window > 1: # 4-dimensional
            uf = func.forecast(u)
            if (i+1+a_window <= na):
                if ft=="ensemble":
                    xa[i+1:i+1+a_window] = np.mean(uf, axis=2)
                    xf[i+1:i+1+a_window] = np.mean(uf, axis=2)
                else:
                    xa[i+1:i+1+a_window] = uf
                    xf[i+1:i+1+a_window] = uf
                ii = 0
                for k in range(i+1,i+1+a_window):
                    if pt=="4dvar":
                        stda[k] = np.sqrt(np.trace(pa)/nx)
                    else:
                        patmp = analysis.calc_pf(uf[ii], pa=pa, cycle=k)
                        stda[k] = np.sqrt(np.trace(patmp)/nx)
                    ii += 1
            else:
                if ft=="ensemble":
                    xa[i+1:na] = np.mean(uf[:na-i-1], axis=2)
                    xf[i+1:na] = np.mean(uf[:na-i-1], axis=2)
                else:
                    xa[i+1:na] = uf[:na-i-1]
                    xf[i+1:na] = uf[:na-i-1]
                ii = 0
                for k in range(i+1,na):
                    if pt=="4dvar":
                        stda[k] = np.sqrt(np.trace(pa)/nx)
                    else:
                        patmp = analysis.calc_pf(uf[ii], pa=pa, cycle=k)
                        stda[k] = np.sqrt(np.trace(patmp)/nx)
                    ii += 1
            u = uf[-1]
            pf = analysis.calc_pf(u, pa=pa, cycle=i+1)
        else:
            u = func.forecast(u)
            pf = analysis.calc_pf(u, pa=pa, cycle=i+1)
        if ft=="ensemble":
            if pt == "mlef" or pt == "4dmlef":
                xf[i+1] = u[:, 0]
            else:
                xf[i+1] = np.mean(u, axis=1)
        else:
            xf[i+1] = u
    # calcurate RMSE and save
    if a_window > 1: # 4-dimensional
        for k in range(i, min(i+a_window,na)):
            e[k] = np.sqrt(np.mean((xa[k, :] - xt[k, :])**2))
    else:
        e[i] = np.sqrt(np.mean((xa[i, :] - xt[i, :])**2))
    stda[i] = np.sqrt(np.trace(pa)/nx)

In [ ]:
obs_s = sigma[op]
x = np.arange(params["na"]) + 1
y = np.ones(x.size) * obs_s
fig, ax = plt.subplots()
ax.plot(x, e, label='RMSE')
ax.plot(x, stda, ls='dashed', label='STDA')
ax.plot(x, y, linestyle="dotted", color="black", label="observation error")
ax.set(xlabel="analysis cycle", title=pt+" "+op)
ax.set_xticks(x[::10])
ax.set_xticks(x[::20],minor=True)
ax.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2,sharex=True,constrained_layout=True)
axs[0].plot(x, chi)
axs[1].plot(x, dfs)
axs[0].set_ylabel(r'$\chi^2$')
axs[1].set_ylabel('DFS')
axs[1].set_xlabel('analysis cycle')
plt.show()